In [ ]:
# !pip uninstall xgboost --y
# !pip install xgboost

# !pip install pytorch_lightning 

In [ ]:
import os
import re
import random
from datetime import datetime

from pathlib import Path
os.chdir('/content/drive/MyDrive/datadriven/airathon')
DATA_PATH = Path.cwd() / 'data'
RAW = DATA_PATH / 'raw'
PROCESSED = DATA_PATH / 'processed3'

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn')
plt.rcParams["figure.figsize"] = (16, 8)

from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, FunctionTransformer, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

import tensorflow as tf
from tensorflow import keras
from keras import layers
import tensorflow_datasets as tfds

In [ ]:
data = pd.read_csv(PROCESSED / 'train_data.csv', parse_dates=['datetime'], index_col=[0]).sort_values('datetime')[0:100]
data.drop_duplicates(subset=['grid_id', 'datetime'], inplace=True)

target = data.value
data.drop(columns=['datetime', 'datetime_dt', 'location', 'date', 'value', 'granule_id'], inplace=True)

In [ ]:
train, test = {}, {}
locations = ['la', 'dl', 'tpe']

for loc in locations:
    train[loc] = pd.read_csv(str(PROCESSED / f'train_{loc}.csv'), parse_dates=['datetime', 'day']).sort_values(by=['datetime'])
    test[loc] = pd.read_csv(str(PROCESSED / f'test_{loc}.csv'), parse_dates=['datetime', 'day']).sort_values(by=['datetime'])
    
data = pd.concat([train['la'], train['dl'], train['tpe']])
test_data = pd.concat([test['la'], test['dl'], test['tpe']])


target = np.array(data.pm25).reshape(-1,1)
target_scaler = MinMaxScaler()
target = target_scaler.fit_transform(target)


data.drop(columns=['datetime', 'pm25', 'day'], axis=1, inplace=True)
test_data.drop(columns=['datetime', 'value', 'day'], axis=1, inplace=True)

ohe_features = ['grid_id']
ohe_transformer = OneHotEncoder(handle_unknown='ignore')

num_features = ['Optical_Depth_047_mean', 'Optical_Depth_047_min', 'Optical_Depth_047_max', 
                'Optical_Depth_055_mean', 'Optical_Depth_055_min', 'Optical_Depth_055_max',
                'AOD_Uncertainty_mean', 'AOD_Uncertainty_min', 'AOD_Uncertainty_max', 
                'Column_WV_mean', 'Column_WV_min', 'Column_WV_max']

num_transformer = Pipeline(steps=[
    ('scaler', MinMaxScaler()),
    ('imputer', SimpleImputer(strategy='median'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_features),
        ('cat', ohe_transformer, ohe_features)])


data = preprocessor.fit_transform(data).toarray()
test_data = preprocessor.transform(test_data).toarray()

n_features = data.shape[1]


In [ ]:
model = keras.Sequential([
    layers.Dense(64, activation=tf.nn.relu, input_shape=[n_features]),
    layers.Dense(64, activation=tf.nn.relu),
    layers.Dense(32, activation=tf.nn.relu),
    layers.Dense(32, activation=tf.nn.relu),
    layers.Dense(16, activation=tf.nn.relu),
    layers.Dense(8, activation=tf.nn.relu),
    layers.Dense(1)
    ])

optimizer = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss='mse', optimizer=optimizer, metrics=['mse'])

In [ ]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 64)                4288      
                                                                 
 dense_25 (Dense)            (None, 64)                4160      
                                                                 
 dense_26 (Dense)            (None, 32)                2080      
                                                                 
 dense_27 (Dense)            (None, 32)                1056      
                                                                 
 dense_28 (Dense)            (None, 16)                528       
                                                                 
 dense_29 (Dense)            (None, 8)                 136       
                                                                 
 dense_30 (Dense)            (None, 1)                

In [ ]:
model.predict(data[0:10])

array([[ 0.00466616],
       [ 0.00855162],
       [ 0.0442536 ],
       [ 0.0128111 ],
       [ 0.00021362],
       [-0.018453  ],
       [ 0.01883634],
       [ 0.01785382],
       [ 0.01125825],
       [-0.00392692]], dtype=float32)

In [ ]:
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0:
            print('')
        print('.', end='')

In [ ]:
EPOCHS = 100
batch_size = 64 

history = model.fit(data, target.reshape(-1,), epochs=EPOCHS, validation_split=0.2, batch_size=batch_size)

Epoch 1/100
257/257 [==============================] - 1s 3ms/step - loss: 0.0025 - mse: 0.0025 - val_loss: 0.0042 - val_mse: 0.0042
Epoch 2/100
257/257 [==============================] - 1s 3ms/step - loss: 0.0019 - mse: 0.0019 - val_loss: 0.0043 - val_mse: 0.0043
Epoch 3/100
257/257 [==============================] - 1s 3ms/step - loss: 0.0018 - mse: 0.0018 - val_loss: 0.0043 - val_mse: 0.0043
Epoch 4/100
257/257 [==============================] - 1s 3ms/step - loss: 0.0018 - mse: 0.0018 - val_loss: 0.0042 - val_mse: 0.0042
Epoch 5/100
257/257 [==============================] - 1s 3ms/step - loss: 0.0018 - mse: 0.0018 - val_loss: 0.0042 - val_mse: 0.0042
Epoch 6/100
257/257 [==============================] - 1s 3ms/step - loss: 0.0018 - mse: 0.0018 - val_loss: 0.0045 - val_mse: 0.0045
Epoch 7/100
257/257 [==============================] - 1s 3ms/step - loss: 0.0018 - mse: 0.0018 - val_loss: 0.0044 - val_mse: 0.0044
Epoch 8/100
257/257 [==============================] - 1s 3ms/step - 

In [ ]:
pred = model.predict(test_data)
pred = target_scaler.inverse_transform(pred)

In [ ]:
test_df = pd.concat([test['la'], test['dl'], test['tpe']])
test_df.value = pred

In [ ]:
submission = pd.read_csv(RAW / 'submission_format.csv', parse_dates=['datetime'])

In [ ]:
for i in tqdm(range(len(submission))):
    sample = submission.iloc[i]
    if len(test_df[(test_df.datetime == sample.datetime) & (test_df.grid_id == sample.grid_id)]) != 1:
        print(len(test_df[(test_df.datetime == sample.datetime) & (test_df.grid_id == sample.grid_id)]))
        break 
        
    submission.loc[i, 'value'] = test_df[(test_df.datetime == sample.datetime) & (test_df.grid_id == sample.grid_id)].value.values[0]

100%|██████████| 13504/13504 [00:46<00:00, 290.55it/s]


In [ ]:
submission_format = pd.read_csv(RAW / 'submission_format.csv')
submission_format.value = submission.value
submission_format.to_csv(DATA_PATH / 'submission/feed_forward.csv', index=False)